In [7]:
import numpy as np
import cupy as cp
import time

def matmul_cpu(A, B):
    return np.dot(A, B)

def matmul_gpu(A, B):
    A_gpu = cp.asarray(A)
    B_gpu = cp.asarray(B)
    C_gpu = cp.dot(A_gpu, B_gpu)
    return cp.asnumpy(C_gpu)

def validate_results(C_cpu, C_gpu):
    return np.allclose(C_cpu, C_gpu, atol=1e-6)

def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

def run_experiments(sizes):
    results = []
    for size in sizes:
        A = np.random.rand(size, size)
        B = np.random.rand(size, size)
        
        C_cpu, time_cpu = measure_time(matmul_cpu, A, B)
        C_gpu, time_gpu = measure_time(matmul_gpu, A, B)
        
        assert validate_results(C_cpu, C_gpu), f"Validation failed for size {size}"
        
        speedup = time_cpu / time_gpu
        results.append((size, time_cpu, time_gpu, speedup))
        print(f"Size: {size}x{size}, CPU Time: {time_cpu:.4f}s, GPU Time: {time_gpu:.4f}s, Speedup: {speedup:.2f}x")
    
    return results


In [8]:
# Размеры матриц для экспериментов
sizes = [100, 500, 1000, 1500, 2000]
results = run_experiments(sizes)

# Сохранение результатов в файл
import csv

with open('results0.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Size", "CPU Time (s)", "GPU Time (s)", "Speedup"])
    writer.writerows(results)

Size: 100x100, CPU Time: 0.0000s, GPU Time: 0.0020s, Speedup: 0.00x
Size: 500x500, CPU Time: 0.0030s, GPU Time: 0.0288s, Speedup: 0.10x
Size: 1000x1000, CPU Time: 0.0104s, GPU Time: 0.1741s, Speedup: 0.06x
Size: 1500x1500, CPU Time: 0.0425s, GPU Time: 0.1019s, Speedup: 0.42x
Size: 2000x2000, CPU Time: 0.0716s, GPU Time: 0.1819s, Speedup: 0.39x
